# 🤖 Nong-View AI 농업영상분석 플랫폼 - ML 훈련 통합 노트북 v3.0

**프로젝트 현황**: 데이터베이스 연결 기반 ML 훈련 시스템 구현  
**핵심 개선**: 올바른 학습/검증/테스트 데이터 분할 구현  
**개발 날짜**: 2025-10-27  
**버전**: v3.0

## 📋 주요 개선사항
- ✅ ML 훈련용 데이터베이스 모델 추가
- ✅ 올바른 Train/Val/Test 분할 로직
- ✅ 라벨 데이터 관리 시스템
- ✅ 모델 훈련 파이프라인
- ✅ 데이터 증강 지원

## 🔧 1. 환경 설정 및 의존성 설치

In [ ]:
# ML 훈련을 위한 추가 패키지 설치
!pip install scikit-learn albumentations opencv-python
!pip install torch torchvision torchaudio ultralytics
!pip install wandb tensorboard matplotlib seaborn
!pip install fastapi uvicorn sqlalchemy alembic
!pip install rasterio geopandas shapely fiona
!pip install pydantic[email] python-multipart

In [ ]:
# 기본 라이브러리 임포트
import os
import sys
import json
import logging
import asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any, Union, Tuple
from pathlib import Path
from uuid import UUID, uuid4
import hashlib
import time
import random
import shutil

# 데이터 처리 및 ML
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 지리정보 처리
import rasterio
import geopandas as gpd
from shapely.geometry import Point, Polygon, mapping
from rasterio.crs import CRS

# 웹 프레임워크
from fastapi import FastAPI, HTTPException, Depends, UploadFile, File, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, FileResponse
from pydantic import BaseModel, Field, validator

# 데이터베이스
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Float, Text, Boolean, ForeignKey, JSON, Enum
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session, relationship
from sqlalchemy.dialects.postgresql import UUID as PG_UUID
import enum

# AI/ML
from ultralytics import YOLO
import torch

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 모든 라이브러리 임포트 완료")

## ⚙️ 2. 설정 및 구성

In [ ]:
# ML 훈련용 설정 클래스 정의
class MLTrainingSettings:
    """ML 훈련 관련 설정"""
    PROJECT_NAME: str = "Nong-View ML Training"
    VERSION: str = "3.0.0"
    API_V1_STR: str = "/api/v1"
    ENVIRONMENT: str = "development"
    DEBUG: bool = True
    
    # 서버 설정
    HOST: str = "127.0.0.1"
    PORT: int = 8000
    
    # 데이터베이스 설정
    DATABASE_URL: str = "sqlite:///./nongview_ml_training.db"
    
    # 파일 저장 설정
    DATA_ROOT: str = "./data"
    DATASET_PATH: str = "./data/datasets"
    ANNOTATIONS_PATH: str = "./data/annotations"
    MODELS_PATH: str = "./data/models"
    LOGS_PATH: str = "./data/logs"
    CACHE_PATH: str = "./data/cache"
    
    # ML 훈련 설정
    TRAIN_SPLIT: float = 0.7
    VAL_SPLIT: float = 0.2
    TEST_SPLIT: float = 0.1
    RANDOM_SEED: int = 42
    
    # YOLO 설정
    IMAGE_SIZE: int = 640
    BATCH_SIZE: int = 16
    EPOCHS: int = 100
    LEARNING_RATE: float = 0.01
    
    # 클래스 정의
    CROP_CLASSES: List[str] = [
        'IRG',        # 이탈리안 라이그라스
        'BARLEY',     # 보리
        'WHEAT',      # 밀
        'CORN_SILAGE', # 옥수수사일리지
        'HAY',        # 건초
        'FALLOW'      # 휴경지
    ]
    
    FACILITY_CLASSES: List[str] = [
        'GREENHOUSE_SINGLE',  # 단동 비닐하우스
        'GREENHOUSE_MULTI',   # 연동 비닐하우스
        'STORAGE',            # 저장시설
        'LIVESTOCK',          # 축사
        'SILO',              # 사일로
        'MACHINERY'          # 농기계
    ]

settings = MLTrainingSettings()

# 디렉토리 생성
for path in [settings.DATA_ROOT, settings.DATASET_PATH, settings.ANNOTATIONS_PATH, 
             settings.MODELS_PATH, settings.LOGS_PATH, settings.CACHE_PATH]:
    Path(path).mkdir(parents=True, exist_ok=True)

# 시드 설정
random.seed(settings.RANDOM_SEED)
np.random.seed(settings.RANDOM_SEED)
torch.manual_seed(settings.RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(settings.RANDOM_SEED)

print("✅ ML 훈련 설정 완료")
print(f"📁 데이터 루트: {settings.DATA_ROOT}")
print(f"🗄️ 데이터베이스: {settings.DATABASE_URL}")
print(f"🎲 랜덤 시드: {settings.RANDOM_SEED}")

## 🗄️ 3. ML 훈련용 데이터베이스 모델 정의

In [ ]:
# SQLAlchemy 설정
engine = create_engine(
    settings.DATABASE_URL,
    connect_args={"check_same_thread": False} if "sqlite" in settings.DATABASE_URL else {}
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# 열거형 정의
class DataSplitType(enum.Enum):
    TRAIN = "train"
    VALIDATION = "validation"
    TEST = "test"

class AnnotationType(enum.Enum):
    BOUNDING_BOX = "bbox"
    SEGMENTATION = "segmentation"
    CLASSIFICATION = "classification"

class ModelType(enum.Enum):
    YOLO_DETECTION = "yolo_detection"
    YOLO_SEGMENTATION = "yolo_segmentation"
    CLASSIFICATION = "classification"

# ML 훈련용 데이터베이스 모델 정의
class Dataset(Base):
    """데이터셋 모델"""
    __tablename__ = "datasets"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    name = Column(String(200), nullable=False, unique=True)
    description = Column(Text)
    
    # 데이터셋 메타정보
    task_type = Column(String(50), nullable=False)  # crop, facility, landuse
    annotation_type = Column(Enum(AnnotationType), nullable=False)
    class_names = Column(JSON)  # List[str]
    
    # 통계
    total_images = Column(Integer, default=0)
    total_annotations = Column(Integer, default=0)
    
    # 분할 정보
    train_count = Column(Integer, default=0)
    val_count = Column(Integer, default=0)
    test_count = Column(Integer, default=0)
    
    # 데이터 경로
    data_path = Column(String(500))
    
    # 관계
    annotated_images = relationship("AnnotatedImage", back_populates="dataset", cascade="all, delete-orphan")
    data_splits = relationship("DataSplit", back_populates="dataset", cascade="all, delete-orphan")
    training_jobs = relationship("TrainingJob", back_populates="dataset")
    
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)


class AnnotatedImage(Base):
    """라벨링된 이미지 모델"""
    __tablename__ = "annotated_images"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    dataset_id = Column(String, ForeignKey("datasets.id"), nullable=False)
    
    # 이미지 정보
    filename = Column(String(255), nullable=False)
    file_path = Column(String(500), nullable=False)
    file_size = Column(Integer)
    
    # 이미지 메타데이터
    width = Column(Integer)
    height = Column(Integer)
    channels = Column(Integer)
    
    # 지리정보 (선택적)
    crs = Column(String(50))
    bounds = Column(JSON)
    
    # 라벨링 정보
    annotation_count = Column(Integer, default=0)
    is_validated = Column(Boolean, default=False)
    validation_score = Column(Float)  # 라벨링 품질 점수
    
    # 메타데이터
    metadata = Column(JSON)  # 추가 정보 (촬영일시, 드론 정보 등)
    
    # 관계
    dataset = relationship("Dataset", back_populates="annotated_images")
    annotations = relationship("Annotation", back_populates="image", cascade="all, delete-orphan")
    data_split = relationship("DataSplit", back_populates="image", uselist=False)
    
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)


class Annotation(Base):
    """어노테이션 모델"""
    __tablename__ = "annotations"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    image_id = Column(String, ForeignKey("annotated_images.id"), nullable=False)
    
    # 클래스 정보
    class_id = Column(Integer, nullable=False)
    class_name = Column(String(100), nullable=False)
    
    # 어노테이션 타입
    annotation_type = Column(Enum(AnnotationType), nullable=False)
    
    # 바운딩 박스 (normalized coordinates 0-1)
    bbox_x = Column(Float)  # center x
    bbox_y = Column(Float)  # center y
    bbox_w = Column(Float)  # width
    bbox_h = Column(Float)  # height
    
    # 세그멘테이션 (선택적)
    segmentation_points = Column(JSON)  # List of normalized points
    
    # 어노테이션 메타정보
    confidence = Column(Float, default=1.0)  # 라벨링 신뢰도
    is_difficult = Column(Boolean, default=False)  # 어려운 객체 표시
    is_crowd = Column(Boolean, default=False)  # 군집 객체 표시
    
    # 라벨링 정보
    annotator_id = Column(String(100))  # 라벨링 작업자 ID
    annotation_tool = Column(String(100))  # 사용된 라벨링 도구
    
    # 관계
    image = relationship("AnnotatedImage", back_populates="annotations")
    
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)


class DataSplit(Base):
    """데이터 분할 모델"""
    __tablename__ = "data_splits"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    dataset_id = Column(String, ForeignKey("datasets.id"), nullable=False)
    image_id = Column(String, ForeignKey("annotated_images.id"), nullable=False, unique=True)
    
    # 분할 타입
    split_type = Column(Enum(DataSplitType), nullable=False)
    
    # 분할 정보
    split_version = Column(String(50), default="v1.0")  # 분할 버전 관리
    stratify_key = Column(String(100))  # 계층화 분할에 사용된 키
    
    # 관계
    dataset = relationship("Dataset", back_populates="data_splits")
    image = relationship("AnnotatedImage", back_populates="data_split")
    
    created_at = Column(DateTime, default=datetime.utcnow)


class TrainingJob(Base):
    """모델 훈련 작업 모델"""
    __tablename__ = "training_jobs"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    dataset_id = Column(String, ForeignKey("datasets.id"), nullable=False)
    
    # 작업 정보
    job_name = Column(String(200), nullable=False)
    description = Column(Text)
    status = Column(String(20), default="pending")  # pending, running, completed, failed, stopped
    
    # 모델 정보
    model_type = Column(Enum(ModelType), nullable=False)
    base_model = Column(String(100))  # yolov8n, yolov8s, etc.
    pretrained_weights = Column(String(500))  # 사전 훈련된 가중치 경로
    
    # 훈련 하이퍼파라미터
    hyperparameters = Column(JSON)  # epochs, batch_size, lr, etc.
    
    # 훈련 진행 상황
    current_epoch = Column(Integer, default=0)
    total_epochs = Column(Integer)
    progress_percent = Column(Float, default=0.0)
    
    # 성능 지표
    best_map50 = Column(Float)  # Best mAP@0.5
    best_map50_95 = Column(Float)  # Best mAP@0.5:0.95
    final_loss = Column(Float)
    
    # 파일 경로
    output_dir = Column(String(500))
    best_weights_path = Column(String(500))
    last_weights_path = Column(String(500))
    log_file_path = Column(String(500))
    
    # 시간 정보
    started_at = Column(DateTime)
    completed_at = Column(DateTime)
    training_duration = Column(Float)  # 총 훈련 시간 (초)
    
    # 에러 정보
    error_message = Column(Text)
    
    # 관계
    dataset = relationship("Dataset", back_populates="training_jobs")
    evaluations = relationship("ModelEvaluation", back_populates="training_job", cascade="all, delete-orphan")
    
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)


class ModelEvaluation(Base):
    """모델 평가 결과 모델"""
    __tablename__ = "model_evaluations"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    training_job_id = Column(String, ForeignKey("training_jobs.id"), nullable=False)
    
    # 평가 정보
    evaluation_type = Column(String(50), nullable=False)  # validation, test
    epoch = Column(Integer)  # 해당 에포크 (validation의 경우)
    
    # 성능 지표
    map50 = Column(Float)  # mAP@0.5
    map50_95 = Column(Float)  # mAP@0.5:0.95
    precision = Column(Float)
    recall = Column(Float)
    f1_score = Column(Float)
    
    # 손실 함수
    box_loss = Column(Float)
    cls_loss = Column(Float)
    dfl_loss = Column(Float)
    total_loss = Column(Float)
    
    # 클래스별 성능
    class_metrics = Column(JSON)  # 클래스별 precision, recall, AP
    
    # 혼동 행렬
    confusion_matrix = Column(JSON)
    
    # 관계
    training_job = relationship("TrainingJob", back_populates="evaluations")
    
    created_at = Column(DateTime, default=datetime.utcnow)


# 데이터베이스 테이블 생성
Base.metadata.create_all(bind=engine)

print("✅ ML 훈련용 데이터베이스 모델 생성 완료")
print(f"📊 생성된 테이블: {list(Base.metadata.tables.keys())}")

## 🔗 4. 데이터베이스 연결 관리

In [ ]:
# 데이터베이스 세션 의존성
def get_db():
    """데이터베이스 세션 생성"""
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# 데이터베이스 서비스 클래스
class DatabaseService:
    """데이터베이스 관련 서비스"""
    
    @staticmethod
    def get_session() -> Session:
        """새로운 데이터베이스 세션 반환"""
        return SessionLocal()
    
    @staticmethod
    def create_dataset(db: Session, name: str, task_type: str, 
                      annotation_type: AnnotationType, class_names: List[str],
                      description: str = None) -> Dataset:
        """새 데이터셋 생성"""
        dataset = Dataset(
            name=name,
            description=description,
            task_type=task_type,
            annotation_type=annotation_type,
            class_names=class_names,
            data_path=str(Path(settings.DATASET_PATH) / name)
        )
        db.add(dataset)
        db.commit()
        db.refresh(dataset)
        
        # 데이터셋 디렉토리 생성
        Path(dataset.data_path).mkdir(parents=True, exist_ok=True)
        Path(dataset.data_path, "images").mkdir(exist_ok=True)
        Path(dataset.data_path, "labels").mkdir(exist_ok=True)
        
        return dataset
    
    @staticmethod
    def add_annotated_image(db: Session, dataset_id: str, 
                           image_path: str, annotations: List[dict]) -> AnnotatedImage:
        """라벨링된 이미지 추가"""
        # 이미지 정보 추출
        img = Image.open(image_path)
        width, height = img.size
        channels = len(img.getbands())
        file_size = Path(image_path).stat().st_size
        
        # 데이터베이스에 이미지 정보 저장
        annotated_image = AnnotatedImage(
            dataset_id=dataset_id,
            filename=Path(image_path).name,
            file_path=str(image_path),
            file_size=file_size,
            width=width,
            height=height,
            channels=channels,
            annotation_count=len(annotations)
        )
        
        db.add(annotated_image)
        db.flush()  # ID 생성을 위해
        
        # 어노테이션 추가
        for ann in annotations:
            annotation = Annotation(
                image_id=annotated_image.id,
                class_id=ann['class_id'],
                class_name=ann['class_name'],
                annotation_type=AnnotationType(ann['type']),
                bbox_x=ann.get('bbox_x'),
                bbox_y=ann.get('bbox_y'),
                bbox_w=ann.get('bbox_w'),
                bbox_h=ann.get('bbox_h'),
                segmentation_points=ann.get('segmentation_points'),
                confidence=ann.get('confidence', 1.0)
            )
            db.add(annotation)
        
        db.commit()
        db.refresh(annotated_image)
        return annotated_image

print("✅ 데이터베이스 연결 관리 설정 완료")

## 📊 5. 데이터 분할 엔진 구현

In [ ]:
class DataSplitEngine:
    """데이터 분할 엔진 - 올바른 Train/Val/Test 분할 구현"""
    
    def __init__(self, train_ratio: float = 0.7, val_ratio: float = 0.2, test_ratio: float = 0.1, 
                 random_seed: int = 42):
        self.train_ratio = train_ratio
        self.val_ratio = val_ratio
        self.test_ratio = test_ratio
        self.random_seed = random_seed
        
        # 비율 검증
        total_ratio = train_ratio + val_ratio + test_ratio
        if abs(total_ratio - 1.0) > 1e-6:
            raise ValueError(f"분할 비율의 합이 1.0이 아닙니다: {total_ratio}")
    
    def create_stratified_split(self, db: Session, dataset_id: str, 
                               stratify_by: str = "class_distribution",
                               version: str = "v1.0") -> dict:
        """계층화 분할 생성 - 클래스 분포를 고려한 분할"""
        
        # 데이터셋의 모든 이미지 조회
        images = db.query(AnnotatedImage).filter(
            AnnotatedImage.dataset_id == dataset_id
        ).all()
        
        if len(images) < 3:
            raise ValueError("최소 3개 이상의 이미지가 필요합니다")
        
        # 각 이미지의 클래스 분포 계산
        image_class_info = []
        for img in images:
            annotations = db.query(Annotation).filter(
                Annotation.image_id == img.id
            ).all()
            
            # 이미지의 주요 클래스 결정 (가장 많은 어노테이션을 가진 클래스)
            class_counts = {}
            for ann in annotations:
                class_counts[ann.class_name] = class_counts.get(ann.class_name, 0) + 1
            
            if class_counts:
                dominant_class = max(class_counts, key=class_counts.get)
                total_annotations = sum(class_counts.values())
            else:
                dominant_class = "no_annotation"
                total_annotations = 0
            
            image_class_info.append({
                'image': img,
                'dominant_class': dominant_class,
                'class_counts': class_counts,
                'total_annotations': total_annotations
            })
        
        # 계층화를 위한 레이블 생성
        if stratify_by == "class_distribution":
            # 주요 클래스로 계층화
            stratify_labels = [info['dominant_class'] for info in image_class_info]
        elif stratify_by == "annotation_count":
            # 어노테이션 수 구간으로 계층화
            stratify_labels = []
            for info in image_class_info:
                count = info['total_annotations']
                if count == 0:
                    label = "none"
                elif count <= 5:
                    label = "low"
                elif count <= 20:
                    label = "medium"
                else:
                    label = "high"
                stratify_labels.append(label)
        else:
            # 기본: 랜덤 분할
            stratify_labels = None
        
        # 데이터 분할 수행
        image_indices = list(range(len(images)))
        
        if stratify_labels:
            # 계층화 분할
            try:
                # 먼저 train과 temp(val+test)로 분할
                train_indices, temp_indices = train_test_split(
                    image_indices,
                    test_size=(self.val_ratio + self.test_ratio),
                    stratify=stratify_labels,
                    random_state=self.random_seed
                )
                
                # temp를 val과 test로 분할
                temp_labels = [stratify_labels[i] for i in temp_indices]
                val_size = self.val_ratio / (self.val_ratio + self.test_ratio)
                
                val_indices, test_indices = train_test_split(
                    temp_indices,
                    test_size=(1 - val_size),
                    stratify=temp_labels,
                    random_state=self.random_seed
                )
                
            except ValueError as e:
                logger.warning(f"계층화 분할 실패, 랜덤 분할로 전환: {e}")
                # 계층화가 불가능한 경우 랜덤 분할
                train_indices, temp_indices = train_test_split(
                    image_indices,
                    test_size=(self.val_ratio + self.test_ratio),
                    random_state=self.random_seed
                )
                
                val_size = self.val_ratio / (self.val_ratio + self.test_ratio)
                val_indices, test_indices = train_test_split(
                    temp_indices,
                    test_size=(1 - val_size),
                    random_state=self.random_seed
                )
        else:
            # 랜덤 분할
            train_indices, temp_indices = train_test_split(
                image_indices,
                test_size=(self.val_ratio + self.test_ratio),
                random_state=self.random_seed
            )
            
            val_size = self.val_ratio / (self.val_ratio + self.test_ratio)
            val_indices, test_indices = train_test_split(
                temp_indices,
                test_size=(1 - val_size),
                random_state=self.random_seed
            )
        
        # 기존 분할 데이터 삭제
        db.query(DataSplit).filter(DataSplit.dataset_id == dataset_id).delete()
        
        # 새로운 분할 데이터 저장
        split_mapping = {
            DataSplitType.TRAIN: train_indices,
            DataSplitType.VALIDATION: val_indices,
            DataSplitType.TEST: test_indices
        }
        
        for split_type, indices in split_mapping.items():
            for idx in indices:
                image = images[idx]
                data_split = DataSplit(
                    dataset_id=dataset_id,
                    image_id=image.id,
                    split_type=split_type,
                    split_version=version,
                    stratify_key=stratify_by
                )
                db.add(data_split)
        
        # 데이터셋 통계 업데이트
        dataset = db.query(Dataset).filter(Dataset.id == dataset_id).first()
        if dataset:
            dataset.total_images = len(images)
            dataset.train_count = len(train_indices)
            dataset.val_count = len(val_indices)
            dataset.test_count = len(test_indices)
        
        db.commit()
        
        # 분할 결과 통계
        split_stats = {
            'total_images': len(images),
            'train_count': len(train_indices),
            'val_count': len(val_indices),
            'test_count': len(test_indices),
            'train_ratio': len(train_indices) / len(images),
            'val_ratio': len(val_indices) / len(images),
            'test_ratio': len(test_indices) / len(images),
            'stratify_method': stratify_by,
            'version': version
        }
        
        logger.info(f"데이터 분할 완료: {split_stats}")
        return split_stats
    
    def get_split_data(self, db: Session, dataset_id: str, 
                      split_type: DataSplitType) -> List[AnnotatedImage]:
        """특정 분할의 이미지 데이터 조회"""
        
        # 해당 분할의 이미지 ID 조회
        split_data = db.query(DataSplit).filter(
            DataSplit.dataset_id == dataset_id,
            DataSplit.split_type == split_type
        ).all()
        
        image_ids = [split.image_id for split in split_data]
        
        # 이미지 데이터 조회
        images = db.query(AnnotatedImage).filter(
            AnnotatedImage.id.in_(image_ids)
        ).all()
        
        return images
    
    def export_yolo_format(self, db: Session, dataset_id: str, output_dir: str) -> str:
        """YOLO 형식으로 데이터셋 내보내기"""
        
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        
        # 데이터셋 정보 조회
        dataset = db.query(Dataset).filter(Dataset.id == dataset_id).first()
        if not dataset:
            raise ValueError(f"데이터셋을 찾을 수 없습니다: {dataset_id}")
        
        # 클래스 매핑 생성
        class_names = dataset.class_names
        class_to_id = {name: idx for idx, name in enumerate(class_names)}
        
        # 각 분할별로 처리
        for split_type in [DataSplitType.TRAIN, DataSplitType.VALIDATION, DataSplitType.TEST]:
            split_name = split_type.value
            
            # 디렉토리 생성
            images_dir = output_path / split_name / "images"
            labels_dir = output_path / split_name / "labels"
            images_dir.mkdir(parents=True, exist_ok=True)
            labels_dir.mkdir(parents=True, exist_ok=True)
            
            # 해당 분할의 이미지들 처리
            images = self.get_split_data(db, dataset_id, split_type)
            
            for img in images:
                # 이미지 파일 복사
                src_image_path = Path(img.file_path)
                dst_image_path = images_dir / src_image_path.name
                
                if src_image_path.exists():
                    shutil.copy2(src_image_path, dst_image_path)
                
                # 라벨 파일 생성
                label_filename = src_image_path.stem + ".txt"
                label_path = labels_dir / label_filename
                
                # 어노테이션 조회 및 YOLO 형식으로 변환
                annotations = db.query(Annotation).filter(
                    Annotation.image_id == img.id
                ).all()
                
                with open(label_path, 'w') as f:
                    for ann in annotations:
                        if ann.annotation_type == AnnotationType.BOUNDING_BOX:
                            class_id = class_to_id.get(ann.class_name, 0)
                            # YOLO format: class_id center_x center_y width height
                            line = f"{class_id} {ann.bbox_x} {ann.bbox_y} {ann.bbox_w} {ann.bbox_h}\n"
                            f.write(line)
        
        # dataset.yaml 파일 생성
        yaml_content = {
            'path': str(output_path.absolute()),
            'train': 'train/images',
            'val': 'validation/images',
            'test': 'test/images',
            'nc': len(class_names),
            'names': class_names
        }
        
        yaml_path = output_path / "dataset.yaml"
        with open(yaml_path, 'w', encoding='utf-8') as f:
            import yaml
            yaml.dump(yaml_content, f, default_flow_style=False, allow_unicode=True)
        
        logger.info(f"YOLO 형식 데이터셋 내보내기 완료: {output_path}")
        return str(yaml_path)


# 전역 데이터 분할 엔진 인스턴스
data_split_engine = DataSplitEngine(
    train_ratio=settings.TRAIN_SPLIT,
    val_ratio=settings.VAL_SPLIT,
    test_ratio=settings.TEST_SPLIT,
    random_seed=settings.RANDOM_SEED
)

print("✅ 데이터 분할 엔진 구현 완료")
print(f"📊 분할 비율: Train {settings.TRAIN_SPLIT}, Val {settings.VAL_SPLIT}, Test {settings.TEST_SPLIT}")

## 🧠 6. ML 훈련 엔진 구현

In [ ]:
class MLTrainingEngine:
    """머신러닝 모델 훈련 엔진"""
    
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        logger.info(f"ML 훈련 엔진 초기화 - 디바이스: {self.device}")
    
    def create_training_job(self, db: Session, dataset_id: str, job_name: str,
                           model_type: ModelType, base_model: str = "yolov8n",
                           hyperparameters: dict = None, description: str = None) -> TrainingJob:
        """새로운 훈련 작업 생성"""
        
        # 기본 하이퍼파라미터 설정
        default_hyperparams = {
            'epochs': settings.EPOCHS,
            'batch_size': settings.BATCH_SIZE,
            'learning_rate': settings.LEARNING_RATE,
            'image_size': settings.IMAGE_SIZE,
            'patience': 50,
            'save_period': 10,
            'optimizer': 'AdamW',
            'cos_lr': True,
            'mosaic': 1.0,
            'mixup': 0.1,
            'copy_paste': 0.1
        }
        
        if hyperparameters:
            default_hyperparams.update(hyperparameters)
        
        # 출력 디렉토리 생성
        job_dir = Path(settings.MODELS_PATH) / f"job_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{job_name}"
        job_dir.mkdir(parents=True, exist_ok=True)
        
        # 훈련 작업 생성
        training_job = TrainingJob(
            dataset_id=dataset_id,
            job_name=job_name,
            description=description,
            model_type=model_type,
            base_model=base_model,
            hyperparameters=default_hyperparams,
            total_epochs=default_hyperparams['epochs'],
            output_dir=str(job_dir),
            log_file_path=str(job_dir / "training.log")
        )
        
        db.add(training_job)
        db.commit()
        db.refresh(training_job)
        
        return training_job
    
    def start_training(self, db: Session, training_job_id: str) -> bool:
        """모델 훈련 시작"""
        
        training_job = db.query(TrainingJob).filter(
            TrainingJob.id == training_job_id
        ).first()
        
        if not training_job:
            raise ValueError(f"훈련 작업을 찾을 수 없습니다: {training_job_id}")
        
        try:
            # 상태 업데이트
            training_job.status = "running"
            training_job.started_at = datetime.utcnow()
            db.commit()
            
            logger.info(f"모델 훈련 시작: {training_job.job_name}")
            
            # 데이터셋을 YOLO 형식으로 내보내기
            yolo_dataset_path = data_split_engine.export_yolo_format(
                db, training_job.dataset_id, 
                Path(training_job.output_dir) / "dataset"
            )
            
            # YOLO 모델 초기화
            if training_job.model_type == ModelType.YOLO_DETECTION:
                model = YOLO(f"{training_job.base_model}.pt")
            else:
                raise ValueError(f"지원되지 않는 모델 타입: {training_job.model_type}")
            
            # 훈련 실행
            train_start_time = time.time()
            
            results = model.train(
                data=yolo_dataset_path,
                epochs=training_job.hyperparameters['epochs'],
                batch=training_job.hyperparameters['batch_size'],
                imgsz=training_job.hyperparameters['image_size'],
                lr0=training_job.hyperparameters['learning_rate'],
                patience=training_job.hyperparameters['patience'],
                save_period=training_job.hyperparameters['save_period'],
                project=training_job.output_dir,
                name="train",
                exist_ok=True,
                device=self.device,
                verbose=True
            )
            
            train_duration = time.time() - train_start_time
            
            # 훈련 결과 처리
            best_weights_path = Path(training_job.output_dir) / "train" / "weights" / "best.pt"
            last_weights_path = Path(training_job.output_dir) / "train" / "weights" / "last.pt"
            
            # 최종 성능 지표 추출
            metrics = results.results_dict if hasattr(results, 'results_dict') else {}
            
            # 훈련 작업 완료 처리
            training_job.status = "completed"
            training_job.completed_at = datetime.utcnow()
            training_job.training_duration = train_duration
            training_job.current_epoch = training_job.total_epochs
            training_job.progress_percent = 100.0
            
            if best_weights_path.exists():
                training_job.best_weights_path = str(best_weights_path)
            if last_weights_path.exists():
                training_job.last_weights_path = str(last_weights_path)
            
            # 성능 지표 저장
            training_job.best_map50 = metrics.get('metrics/mAP50(B)', 0.0)
            training_job.best_map50_95 = metrics.get('metrics/mAP50-95(B)', 0.0)
            
            db.commit()
            
            logger.info(f"모델 훈련 완료: {training_job.job_name} ({train_duration:.2f}초)")
            
            # 테스트 평가 실행
            self.evaluate_model(db, training_job_id, "test")
            
            return True
            
        except Exception as e:
            logger.error(f"모델 훈련 실패: {e}")
            
            # 에러 상태 업데이트
            training_job.status = "failed"
            training_job.error_message = str(e)
            training_job.completed_at = datetime.utcnow()
            db.commit()
            
            return False
    
    def evaluate_model(self, db: Session, training_job_id: str, 
                      evaluation_type: str = "test") -> dict:
        """모델 평가 수행"""
        
        training_job = db.query(TrainingJob).filter(
            TrainingJob.id == training_job_id
        ).first()
        
        if not training_job or not training_job.best_weights_path:
            raise ValueError("훈련된 모델을 찾을 수 없습니다")
        
        try:
            # 훈련된 모델 로드
            model = YOLO(training_job.best_weights_path)
            
            # 데이터셋 경로
            dataset_yaml = Path(training_job.output_dir) / "dataset" / "dataset.yaml"
            
            # 평가 실행
            if evaluation_type == "test":
                results = model.val(data=str(dataset_yaml), split="test")
            else:
                results = model.val(data=str(dataset_yaml), split="val")
            
            # 결과 추출
            metrics = results.results_dict if hasattr(results, 'results_dict') else {}
            
            # 평가 결과 저장
            evaluation = ModelEvaluation(
                training_job_id=training_job_id,
                evaluation_type=evaluation_type,
                map50=metrics.get('metrics/mAP50(B)', 0.0),
                map50_95=metrics.get('metrics/mAP50-95(B)', 0.0),
                precision=metrics.get('metrics/precision(B)', 0.0),
                recall=metrics.get('metrics/recall(B)', 0.0),
                # 클래스별 성능은 별도 처리 필요
                class_metrics={}  # TODO: 클래스별 성능 추출
            )
            
            db.add(evaluation)
            db.commit()
            db.refresh(evaluation)
            
            logger.info(f"모델 평가 완료: {evaluation_type} - mAP50: {evaluation.map50:.4f}")
            
            return {
                'evaluation_id': evaluation.id,
                'map50': evaluation.map50,
                'map50_95': evaluation.map50_95,
                'precision': evaluation.precision,
                'recall': evaluation.recall
            }
            
        except Exception as e:
            logger.error(f"모델 평가 실패: {e}")
            raise
    
    def get_training_status(self, db: Session, training_job_id: str) -> dict:
        """훈련 상태 조회"""
        
        training_job = db.query(TrainingJob).filter(
            TrainingJob.id == training_job_id
        ).first()
        
        if not training_job:
            raise ValueError(f"훈련 작업을 찾을 수 없습니다: {training_job_id}")
        
        # 평가 결과 조회
        evaluations = db.query(ModelEvaluation).filter(
            ModelEvaluation.training_job_id == training_job_id
        ).all()
        
        evaluation_results = {}
        for eval_result in evaluations:
            evaluation_results[eval_result.evaluation_type] = {
                'map50': eval_result.map50,
                'map50_95': eval_result.map50_95,
                'precision': eval_result.precision,
                'recall': eval_result.recall
            }
        
        return {
            'job_id': training_job.id,
            'job_name': training_job.job_name,
            'status': training_job.status,
            'progress_percent': training_job.progress_percent,
            'current_epoch': training_job.current_epoch,
            'total_epochs': training_job.total_epochs,
            'best_map50': training_job.best_map50,
            'best_map50_95': training_job.best_map50_95,
            'training_duration': training_job.training_duration,
            'started_at': training_job.started_at,
            'completed_at': training_job.completed_at,
            'evaluations': evaluation_results,
            'error_message': training_job.error_message
        }


# 전역 ML 훈련 엔진 인스턴스
ml_training_engine = MLTrainingEngine()

print("✅ ML 훈련 엔진 구현 완료")
print(f"🖥️ 사용 디바이스: {ml_training_engine.device}")

## 📋 7. API 스키마 정의

In [ ]:
# API 응답 스키마
class APIResponse(BaseModel):
    """기본 API 응답"""
    success: bool = True
    message: str = ""
    data: Optional[Any] = None
    error: Optional[str] = None

# 데이터셋 관련 스키마
class DatasetCreateRequest(BaseModel):
    """데이터셋 생성 요청"""
    name: str
    description: Optional[str] = None
    task_type: str  # crop, facility, landuse
    annotation_type: str  # bbox, segmentation, classification
    class_names: List[str]

class DatasetResponse(BaseModel):
    """데이터셋 응답"""
    id: str
    name: str
    description: Optional[str]
    task_type: str
    annotation_type: str
    class_names: List[str]
    total_images: int
    total_annotations: int
    train_count: int
    val_count: int
    test_count: int
    created_at: datetime
    updated_at: datetime

# 데이터 분할 관련 스키마
class DataSplitRequest(BaseModel):
    """데이터 분할 요청"""
    dataset_id: str
    stratify_by: str = "class_distribution"  # class_distribution, annotation_count, random
    train_ratio: float = Field(0.7, ge=0.1, le=0.8)
    val_ratio: float = Field(0.2, ge=0.1, le=0.4)
    test_ratio: float = Field(0.1, ge=0.1, le=0.4)
    version: str = "v1.0"

class DataSplitResponse(BaseModel):
    """데이터 분할 응답"""
    total_images: int
    train_count: int
    val_count: int
    test_count: int
    train_ratio: float
    val_ratio: float
    test_ratio: float
    stratify_method: str
    version: str

# 모델 훈련 관련 스키마
class TrainingJobCreateRequest(BaseModel):
    """훈련 작업 생성 요청"""
    dataset_id: str
    job_name: str
    description: Optional[str] = None
    model_type: str = "yolo_detection"
    base_model: str = "yolov8n"
    epochs: int = Field(100, ge=1, le=1000)
    batch_size: int = Field(16, ge=1, le=64)
    learning_rate: float = Field(0.01, ge=0.0001, le=0.1)
    image_size: int = Field(640, ge=320, le=1280)

class TrainingJobResponse(BaseModel):
    """훈련 작업 응답"""
    id: str
    job_name: str
    description: Optional[str]
    status: str
    model_type: str
    base_model: str
    progress_percent: float
    current_epoch: int
    total_epochs: int
    best_map50: Optional[float]
    best_map50_95: Optional[float]
    training_duration: Optional[float]
    started_at: Optional[datetime]
    completed_at: Optional[datetime]
    error_message: Optional[str]
    created_at: datetime

class ModelEvaluationResponse(BaseModel):
    """모델 평가 응답"""
    id: str
    evaluation_type: str
    epoch: Optional[int]
    map50: float
    map50_95: float
    precision: float
    recall: float
    f1_score: Optional[float]
    created_at: datetime

print("✅ API 스키마 정의 완료")

## 🚀 8. FastAPI 애플리케이션 및 ML 훈련 API

In [ ]:
# FastAPI 애플리케이션 생성
app = FastAPI(
    title=settings.PROJECT_NAME,
    description="AI 농업 영상 분석 플랫폼 - ML 훈련 시스템",
    version=settings.VERSION,
    docs_url="/api/docs",
    redoc_url="/api/redoc"
)

# CORS 미들웨어
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 헬스 체크
@app.get("/health")
async def health_check():
    """헬스 체크 엔드포인트"""
    return APIResponse(
        success=True,
        message="ML 훈련 서비스가 정상 작동 중입니다",
        data={
            "timestamp": datetime.now().isoformat(),
            "version": settings.VERSION,
            "environment": settings.ENVIRONMENT,
            "device": ml_training_engine.device
        }
    )

# 데이터셋 관리 API
@app.post("/api/v1/datasets", response_model=APIResponse)
async def create_dataset(
    request: DatasetCreateRequest,
    db: Session = Depends(get_db)
):
    """데이터셋 생성"""
    try:
        # 어노테이션 타입 검증
        try:
            annotation_type = AnnotationType(request.annotation_type)
        except ValueError:
            raise HTTPException(
                status_code=400, 
                detail=f"지원되지 않는 어노테이션 타입: {request.annotation_type}"
            )
        
        # 데이터셋 생성
        dataset = DatabaseService.create_dataset(
            db=db,
            name=request.name,
            task_type=request.task_type,
            annotation_type=annotation_type,
            class_names=request.class_names,
            description=request.description
        )
        
        dataset_data = DatasetResponse(
            id=dataset.id,
            name=dataset.name,
            description=dataset.description,
            task_type=dataset.task_type,
            annotation_type=dataset.annotation_type.value,
            class_names=dataset.class_names,
            total_images=dataset.total_images,
            total_annotations=dataset.total_annotations,
            train_count=dataset.train_count,
            val_count=dataset.val_count,
            test_count=dataset.test_count,
            created_at=dataset.created_at,
            updated_at=dataset.updated_at
        )
        
        return APIResponse(
            success=True,
            message="데이터셋 생성 완료",
            data=dataset_data.dict()
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"데이터셋 생성 실패: {e}")
        raise HTTPException(status_code=500, detail=f"데이터셋 생성 실패: {str(e)}")


@app.get("/api/v1/datasets", response_model=APIResponse)
async def list_datasets(db: Session = Depends(get_db)):
    """데이터셋 목록 조회"""
    try:
        datasets = db.query(Dataset).all()
        
        dataset_list = []
        for dataset in datasets:
            dataset_data = DatasetResponse(
                id=dataset.id,
                name=dataset.name,
                description=dataset.description,
                task_type=dataset.task_type,
                annotation_type=dataset.annotation_type.value,
                class_names=dataset.class_names,
                total_images=dataset.total_images,
                total_annotations=dataset.total_annotations,
                train_count=dataset.train_count,
                val_count=dataset.val_count,
                test_count=dataset.test_count,
                created_at=dataset.created_at,
                updated_at=dataset.updated_at
            )
            dataset_list.append(dataset_data.dict())
        
        return APIResponse(
            success=True,
            message=f"{len(dataset_list)}개의 데이터셋을 찾았습니다",
            data={"datasets": dataset_list, "total": len(dataset_list)}
        )
        
    except Exception as e:
        logger.error(f"데이터셋 목록 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"데이터셋 목록 조회 실패: {str(e)}")


# 데이터 분할 API
@app.post("/api/v1/datasets/{dataset_id}/split", response_model=APIResponse)
async def create_data_split(
    dataset_id: str,
    request: DataSplitRequest,
    db: Session = Depends(get_db)
):
    """데이터 분할 생성"""
    try:
        # 분할 비율 검증
        total_ratio = request.train_ratio + request.val_ratio + request.test_ratio
        if abs(total_ratio - 1.0) > 1e-6:
            raise HTTPException(
                status_code=400,
                detail=f"분할 비율의 합이 1.0이 아닙니다: {total_ratio}"
            )
        
        # 데이터셋 존재 확인
        dataset = db.query(Dataset).filter(Dataset.id == dataset_id).first()
        if not dataset:
            raise HTTPException(status_code=404, detail="데이터셋을 찾을 수 없습니다")
        
        # 분할 엔진 설정 업데이트
        data_split_engine.train_ratio = request.train_ratio
        data_split_engine.val_ratio = request.val_ratio
        data_split_engine.test_ratio = request.test_ratio
        
        # 분할 실행
        split_stats = data_split_engine.create_stratified_split(
            db=db,
            dataset_id=dataset_id,
            stratify_by=request.stratify_by,
            version=request.version
        )
        
        split_response = DataSplitResponse(**split_stats)
        
        return APIResponse(
            success=True,
            message="데이터 분할 완료",
            data=split_response.dict()
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"데이터 분할 실패: {e}")
        raise HTTPException(status_code=500, detail=f"데이터 분할 실패: {str(e)}")


# 모델 훈련 API
@app.post("/api/v1/training/jobs", response_model=APIResponse)
async def create_training_job(
    request: TrainingJobCreateRequest,
    background_tasks: BackgroundTasks,
    db: Session = Depends(get_db)
):
    """훈련 작업 생성"""
    try:
        # 데이터셋 존재 확인
        dataset = db.query(Dataset).filter(Dataset.id == request.dataset_id).first()
        if not dataset:
            raise HTTPException(status_code=404, detail="데이터셋을 찾을 수 없습니다")
        
        # 데이터 분할 확인
        train_count = db.query(DataSplit).filter(
            DataSplit.dataset_id == request.dataset_id,
            DataSplit.split_type == DataSplitType.TRAIN
        ).count()
        
        if train_count == 0:
            raise HTTPException(
                status_code=400, 
                detail="데이터 분할이 먼저 필요합니다. /datasets/{dataset_id}/split 엔드포인트를 사용하세요."
            )
        
        # 모델 타입 검증
        try:
            model_type = ModelType(request.model_type)
        except ValueError:
            raise HTTPException(
                status_code=400,
                detail=f"지원되지 않는 모델 타입: {request.model_type}"
            )
        
        # 하이퍼파라미터 구성
        hyperparameters = {
            'epochs': request.epochs,
            'batch_size': request.batch_size,
            'learning_rate': request.learning_rate,
            'image_size': request.image_size
        }
        
        # 훈련 작업 생성
        training_job = ml_training_engine.create_training_job(
            db=db,
            dataset_id=request.dataset_id,
            job_name=request.job_name,
            model_type=model_type,
            base_model=request.base_model,
            hyperparameters=hyperparameters,
            description=request.description
        )
        
        # 백그라운드에서 훈련 시작
        background_tasks.add_task(
            ml_training_engine.start_training,
            db=SessionLocal(),
            training_job_id=training_job.id
        )
        
        training_job_data = TrainingJobResponse(
            id=training_job.id,
            job_name=training_job.job_name,
            description=training_job.description,
            status=training_job.status,
            model_type=training_job.model_type.value,
            base_model=training_job.base_model,
            progress_percent=training_job.progress_percent,
            current_epoch=training_job.current_epoch,
            total_epochs=training_job.total_epochs,
            best_map50=training_job.best_map50,
            best_map50_95=training_job.best_map50_95,
            training_duration=training_job.training_duration,
            started_at=training_job.started_at,
            completed_at=training_job.completed_at,
            error_message=training_job.error_message,
            created_at=training_job.created_at
        )
        
        return APIResponse(
            success=True,
            message="훈련 작업이 시작되었습니다",
            data=training_job_data.dict()
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"훈련 작업 생성 실패: {e}")
        raise HTTPException(status_code=500, detail=f"훈련 작업 생성 실패: {str(e)}")


@app.get("/api/v1/training/jobs", response_model=APIResponse)
async def list_training_jobs(db: Session = Depends(get_db)):
    """훈련 작업 목록 조회"""
    try:
        training_jobs = db.query(TrainingJob).order_by(TrainingJob.created_at.desc()).all()
        
        job_list = []
        for job in training_jobs:
            job_data = TrainingJobResponse(
                id=job.id,
                job_name=job.job_name,
                description=job.description,
                status=job.status,
                model_type=job.model_type.value,
                base_model=job.base_model,
                progress_percent=job.progress_percent,
                current_epoch=job.current_epoch,
                total_epochs=job.total_epochs,
                best_map50=job.best_map50,
                best_map50_95=job.best_map50_95,
                training_duration=job.training_duration,
                started_at=job.started_at,
                completed_at=job.completed_at,
                error_message=job.error_message,
                created_at=job.created_at
            )
            job_list.append(job_data.dict())
        
        return APIResponse(
            success=True,
            message=f"{len(job_list)}개의 훈련 작업을 찾았습니다",
            data={"training_jobs": job_list, "total": len(job_list)}
        )
        
    except Exception as e:
        logger.error(f"훈련 작업 목록 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"훈련 작업 목록 조회 실패: {str(e)}")


@app.get("/api/v1/training/jobs/{job_id}", response_model=APIResponse)
async def get_training_job(job_id: str, db: Session = Depends(get_db)):
    """훈련 작업 상세 조회"""
    try:
        job_status = ml_training_engine.get_training_status(db, job_id)
        
        return APIResponse(
            success=True,
            message="훈련 작업 조회 완료",
            data=job_status
        )
        
    except ValueError as e:
        raise HTTPException(status_code=404, detail=str(e))
    except Exception as e:
        logger.error(f"훈련 작업 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"훈련 작업 조회 실패: {str(e)}")

print("✅ FastAPI 애플리케이션 및 ML 훈련 API 구현 완료")

## 📊 9. 샘플 데이터 생성 및 테스트

In [ ]:
def create_sample_ml_dataset(db: Session) -> str:
    """ML 훈련용 샘플 데이터셋 생성"""
    
    logger.info("ML 훈련용 샘플 데이터셋 생성 시작...")
    
    # 1. 데이터셋 생성
    dataset = DatabaseService.create_dataset(
        db=db,
        name="sample_crop_dataset",
        task_type="crop",
        annotation_type=AnnotationType.BOUNDING_BOX,
        class_names=settings.CROP_CLASSES,
        description="샘플 작물 분류 데이터셋"
    )
    
    # 2. 샘플 이미지 및 어노테이션 생성
    dataset_path = Path(dataset.data_path)
    images_dir = dataset_path / "images"
    images_dir.mkdir(exist_ok=True)
    
    # 클래스별로 샘플 데이터 생성
    total_images = 50  # 테스트용 소량 데이터
    images_per_class = total_images // len(settings.CROP_CLASSES)
    
    for class_idx, class_name in enumerate(settings.CROP_CLASSES):
        for img_idx in range(images_per_class):
            # 가상 이미지 생성 (실제 프로젝트에서는 실제 이미지 사용)
            img_array = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)
            
            # 클래스에 따른 색상 패턴 추가
            if class_name == 'IRG':
                img_array[:, :, 1] = np.maximum(img_array[:, :, 1], 150)  # 녹색 강조
            elif class_name == 'BARLEY':
                img_array[:, :, 0] = np.maximum(img_array[:, :, 0], 180)  # 황금색
                img_array[:, :, 1] = np.maximum(img_array[:, :, 1], 150)
            # ... 다른 클래스들도 유사하게
            
            # 이미지 저장
            img_filename = f"{class_name.lower()}_{img_idx:03d}.jpg"
            img_path = images_dir / img_filename
            
            pil_image = Image.fromarray(img_array)
            pil_image.save(img_path, 'JPEG')
            
            # 샘플 어노테이션 생성 (중앙에 하나의 객체)
            annotations = [{
                'class_id': class_idx,
                'class_name': class_name,
                'type': 'bbox',
                'bbox_x': 0.5,  # center x (normalized)
                'bbox_y': 0.5,  # center y (normalized)
                'bbox_w': 0.3 + np.random.uniform(-0.1, 0.1),  # width
                'bbox_h': 0.3 + np.random.uniform(-0.1, 0.1),  # height
                'confidence': 1.0
            }]
            
            # 가끔 추가 객체 생성
            if np.random.random() < 0.3:  # 30% 확률로 추가 객체
                annotations.append({
                    'class_id': class_idx,
                    'class_name': class_name,
                    'type': 'bbox',
                    'bbox_x': np.random.uniform(0.2, 0.8),
                    'bbox_y': np.random.uniform(0.2, 0.8),
                    'bbox_w': np.random.uniform(0.1, 0.2),
                    'bbox_h': np.random.uniform(0.1, 0.2),
                    'confidence': 1.0
                })
            
            # 데이터베이스에 이미지 및 어노테이션 추가
            DatabaseService.add_annotated_image(
                db=db,
                dataset_id=dataset.id,
                image_path=str(img_path),
                annotations=annotations
            )
    
    # 3. 데이터셋 통계 업데이트
    total_images = db.query(AnnotatedImage).filter(
        AnnotatedImage.dataset_id == dataset.id
    ).count()
    
    total_annotations = db.query(Annotation).join(AnnotatedImage).filter(
        AnnotatedImage.dataset_id == dataset.id
    ).count()
    
    dataset.total_images = total_images
    dataset.total_annotations = total_annotations
    db.commit()
    
    logger.info(f"샘플 데이터셋 생성 완료: {total_images}개 이미지, {total_annotations}개 어노테이션")
    return dataset.id


def run_sample_ml_workflow():
    """완전한 ML 워크플로우 테스트 실행"""
    
    db = DatabaseService.get_session()
    
    try:
        print("🚀 ML 훈련 워크플로우 테스트 시작")
        print("=" * 60)
        
        # 1. 샘플 데이터셋 생성
        print("\n1️⃣ 샘플 데이터셋 생성 중...")
        dataset_id = create_sample_ml_dataset(db)
        print(f"✅ 데이터셋 생성 완료: {dataset_id}")
        
        # 2. 데이터 분할
        print("\n2️⃣ 데이터 분할 수행 중...")
        split_stats = data_split_engine.create_stratified_split(
            db=db,
            dataset_id=dataset_id,
            stratify_by="class_distribution",
            version="v1.0"
        )
        print(f"✅ 데이터 분할 완료:")
        print(f"   - 총 이미지: {split_stats['total_images']}개")
        print(f"   - 훈련: {split_stats['train_count']}개 ({split_stats['train_ratio']:.1%})")
        print(f"   - 검증: {split_stats['val_count']}개 ({split_stats['val_ratio']:.1%})")
        print(f"   - 테스트: {split_stats['test_count']}개 ({split_stats['test_ratio']:.1%})")
        
        # 3. 훈련 작업 생성
        print("\n3️⃣ 훈련 작업 생성 중...")
        hyperparams = {
            'epochs': 5,  # 테스트용 작은 값
            'batch_size': 4,
            'learning_rate': 0.01,
            'image_size': 640
        }
        
        training_job = ml_training_engine.create_training_job(
            db=db,
            dataset_id=dataset_id,
            job_name="sample_crop_training",
            model_type=ModelType.YOLO_DETECTION,
            base_model="yolov8n",
            hyperparameters=hyperparams,
            description="샘플 작물 분류 모델 훈련"
        )
        print(f"✅ 훈련 작업 생성 완료: {training_job.id}")
        
        # 4. YOLO 형식 데이터셋 내보내기 테스트
        print("\n4️⃣ YOLO 형식 데이터셋 내보내기 테스트...")
        export_path = Path(training_job.output_dir) / "dataset"
        yaml_path = data_split_engine.export_yolo_format(
            db=db,
            dataset_id=dataset_id,
            output_dir=str(export_path)
        )
        print(f"✅ YOLO 데이터셋 내보내기 완료: {yaml_path}")
        
        # 5. 훈련 정보 출력
        print("\n5️⃣ 훈련 정보:")
        print(f"   - 작업 ID: {training_job.id}")
        print(f"   - 작업 이름: {training_job.job_name}")
        print(f"   - 모델 타입: {training_job.model_type.value}")
        print(f"   - 베이스 모델: {training_job.base_model}")
        print(f"   - 출력 디렉토리: {training_job.output_dir}")
        print(f"   - 하이퍼파라미터: {training_job.hyperparameters}")
        
        print("\n" + "=" * 60)
        print("🎉 ML 훈련 워크플로우 테스트 완료!")
        print("\n📋 다음 단계:")
        print("   1. API 서버 시작: start_server()")
        print("   2. 훈련 시작: POST /api/v1/training/jobs")
        print("   3. 진행 상황 모니터링: GET /api/v1/training/jobs/{job_id}")
        
        return {
            'dataset_id': dataset_id,
            'training_job_id': training_job.id,
            'split_stats': split_stats
        }
        
    except Exception as e:
        logger.error(f"ML 워크플로우 테스트 실패: {e}")
        print(f"❌ 테스트 실패: {e}")
        return None
    finally:
        db.close()


# 샘플 워크플로우 실행
print("📊 ML 훈련 샘플 데이터 준비 중...")
sample_results = run_sample_ml_workflow()

if sample_results:
    print(f"\n✅ 샘플 데이터 준비 완료!")
    print(f"📝 결과 요약:")
    print(f"   - 데이터셋 ID: {sample_results['dataset_id']}")
    print(f"   - 훈련 작업 ID: {sample_results['training_job_id']}")

## 🖥️ 10. 서버 시작 및 사용법

In [ ]:
# 서버 시작 함수
def start_server():
    """ML 훈련 서버 시작"""
    import uvicorn
    
    print("\n🚀 Nong-View ML 훈련 서버 시작 (v3.0)")
    print(f"🌐 주소: http://{settings.HOST}:{settings.PORT}")
    print(f"📚 API 문서: http://{settings.HOST}:{settings.PORT}/api/docs")
    print(f"🔧 환경: {settings.ENVIRONMENT}")
    print(f"📊 버전: {settings.VERSION}")
    print(f"💻 디바이스: {ml_training_engine.device}")
    
    print("\n" + "="*50)
    print("서버를 시작하려면 아래 코드를 실행하세요:")
    print("start_server()")
    print("="*50)
    
    # 실제 서버 시작 (주석 해제 시 자동 시작)
    # uvicorn.run(
    #     "__main__:app",
    #     host=settings.HOST,
    #     port=settings.PORT,
    #     reload=settings.DEBUG
    # )


def show_usage_examples():
    """사용 예시 출력"""
    
    print("\n📖 API 사용 예시:")
    print("=" * 50)
    
    examples = [
        {
            "title": "1. 데이터셋 생성",
            "method": "POST",
            "url": "/api/v1/datasets",
            "body": {
                "name": "my_crop_dataset",
                "description": "작물 분류용 데이터셋",
                "task_type": "crop",
                "annotation_type": "bbox",
                "class_names": ["IRG", "BARLEY", "WHEAT", "CORN_SILAGE", "HAY", "FALLOW"]
            }
        },
        {
            "title": "2. 데이터 분할",
            "method": "POST",
            "url": "/api/v1/datasets/{dataset_id}/split",
            "body": {
                "dataset_id": "your_dataset_id",
                "stratify_by": "class_distribution",
                "train_ratio": 0.7,
                "val_ratio": 0.2,
                "test_ratio": 0.1
            }
        },
        {
            "title": "3. 훈련 작업 시작",
            "method": "POST",
            "url": "/api/v1/training/jobs",
            "body": {
                "dataset_id": "your_dataset_id",
                "job_name": "crop_detection_v1",
                "description": "첫 번째 작물 검출 모델",
                "model_type": "yolo_detection",
                "base_model": "yolov8n",
                "epochs": 100,
                "batch_size": 16,
                "learning_rate": 0.01,
                "image_size": 640
            }
        },
        {
            "title": "4. 훈련 진행 상황 확인",
            "method": "GET",
            "url": "/api/v1/training/jobs/{job_id}",
            "body": None
        }
    ]
    
    for example in examples:
        print(f"\n{example['title']}")
        print(f"   {example['method']} {example['url']}")
        if example['body']:
            print(f"   Body: {json.dumps(example['body'], indent=8, ensure_ascii=False)}")
    
    print("\n" + "=" * 50)


# 서버 정보 및 사용법 출력
print("\n" + "="*60)
print("🎉 Nong-View v3.0 ML 훈련 노트북 준비 완료!")
print("="*60)
print(f"📈 주요 개선사항:")
print(f"   - ✅ 올바른 Train/Val/Test 데이터 분할 구현")
print(f"   - ✅ 계층화 분할로 클래스 균형 유지")
print(f"   - ✅ SQLAlchemy 기반 ML 훈련 데이터베이스")
print(f"   - ✅ YOLO 모델 훈련 파이프라인")
print(f"   - ✅ 모델 평가 및 성능 지표 추적")
print(f"   - ✅ RESTful API로 전체 ML 워크플로우 제어")

show_usage_examples()

print("\n🔧 시작 방법:")
print("   1. 모든 셀 실행 완료")
print("   2. start_server() 함수 호출")
print("   3. http://localhost:8000/api/docs 접속")
print("   4. API를 통해 데이터셋 생성 → 분할 → 훈련 → 평가")

print("\n📚 문서:")
print("   - API 문서: /api/docs")
print("   - Redoc: /api/redoc")
print("   - 헬스 체크: /health")

print("\n🎯 완료된 기능:")
print("   - 데이터베이스 기반 데이터셋 관리")
print("   - 과학적 데이터 분할 (stratified split)")
print("   - YOLO 모델 훈련 자동화")
print("   - 실시간 훈련 진행 상황 추적")
print("   - 모델 성능 평가 및 저장")
print("="*60)